In [ ]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0,
    openai_api_key=os.getenv("MY_OPENAI_API_KEY")
)
